## Capstone Project - Car accident severity (Week 1)

## Introduction::

#### The Target in this project is to prevent avoidable car accidents by implementing algorithim that alert drivers, health system, and police to remind them to be more careful in critical situations. given the current weather, road and visibility conditions. When conditions are bad, this model will alert drivers to remind them to be more careful, and police rescue teams to be ready for such severe accidents.

#### we want to analyze the accident “severity” in terms of human fatality, traffic delay, property damage, or any other type of accident bad impact. The data was collected by Seattle SPOT Traffic Management Division and provided by Coursera via a link. 


#### This dataset is updated weekly and is from 2004 to present. It contains information such as severity code, address type, location, collision type, weather, road condition, speeding, among others.

### **The main reasons for the accidents::

  1-preventable; not paying enough attention during driving, abusing drugs and alcohol or driving at very high speed.

 2-uncontrolled; weather, visibility, or road conditions

## Data Understanding::

The data was collected by the Seattle Police Department and Accident Traffic Records Department from 2004 to present. The data consists of 37 independent variables and 194,673 rows. The dependent variable, “SEVERITYCODE”, contains numbers that correspond to different levels of severity caused by an accident from 0 to 3.


**A code that corresponds to the severity of the collision:**<br>
• 3 — fatality <br>
• 2b — serious injury <br>
• 2 — injury <br>
• 1 — prop damage<br>
• 0 — unknown <br>

**The attributes i will use to make my model is (ADDRTYPE, WEATHER, ROADCOND, VEHCOUNT, PERSONCOUNT)** <br> and of course will do some data cleaning on the extracted data.


In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import warnings
from sklearn.utils import resample
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [40]:
url = "https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv"
df = pd.read_csv(url)

df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [41]:
df.describe()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,INTKEY,SEVERITYCODE.1,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,SDOT_COLCODE,SDOTCOLNUM,SEGLANEKEY,CROSSWALKKEY
count,194673.000000,189339.000000,189339.000000,194673.000000,194673.000000,194673.000000,65070.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,1.149360e+05,194673.000000,1.946730e+05
mean,1.298901,-122.330518,47.619543,108479.364930,141091.456350,141298.811381,37558.450576,1.298901,2.444427,0.037139,0.028391,1.920780,13.867768,7.972521e+06,269.401114,9.782452e+03
std,0.457778,0.029976,0.056157,62649.722558,86634.402737,86986.542110,51745.990273,0.457778,1.345929,0.198150,0.167413,0.631047,6.868755,2.553533e+06,3315.776055,7.226926e+04
min,1.000000,-122.419091,47.495573,1.000000,1001.000000,1001.000000,23807.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.007024e+06,0.000000,0.000000e+00
25%,1.000000,-122.348673,47.575956,54267.000000,70383.000000,70383.000000,28667.000000,1.000000,2.000000,0.000000,0.000000,2.000000,11.000000,6.040015e+06,0.000000,0.000000e+00
50%,1.000000,-122.330224,47.615369,106912.000000,123363.000000,123363.000000,29973.000000,1.000000,2.000000,0.000000,0.000000,2.000000,13.000000,8.023022e+06,0.000000,0.000000e+00
75%,2.000000,-122.311937,47.663664,162272.000000,203319.000000,203459.000000,33973.000000,2.000000,3.000000,0.000000,0.000000,2.000000,14.000000,1.015501e+07,0.000000,0.000000e+00
max,2.000000,-122.238949,47.734142,219547.000000,331454.000000,332954.000000,757580.000000,2.000000,81.000000,6.000000,2.000000,12.000000,69.000000,1.307202e+07,525241.000000,5.239700e+06


In [42]:
df.shape

(194673, 38)

## print the columns in the dataset to pick the columns data we will use for the model

In [43]:
df.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

## pick the columns from the dataset, we will use in the prediction model;;

In [44]:
df = df.loc[:,['SEVERITYCODE','WEATHER','ROADCOND','LIGHTCOND','VEHCOUNT','ADDRTYPE','PERSONCOUNT']]
df.dropna(inplace=True)
df.head(15)

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND,VEHCOUNT,ADDRTYPE,PERSONCOUNT
0,2,Overcast,Wet,Daylight,2,Intersection,2
1,1,Raining,Wet,Dark - Street Lights On,2,Block,2
2,1,Overcast,Dry,Daylight,3,Block,4
3,1,Clear,Dry,Daylight,3,Block,3
4,2,Raining,Wet,Daylight,2,Intersection,2
5,1,Clear,Dry,Daylight,2,Intersection,2
6,1,Raining,Wet,Daylight,2,Intersection,2
7,2,Clear,Dry,Daylight,1,Intersection,3
8,1,Clear,Dry,Daylight,2,Block,2
9,2,Clear,Dry,Daylight,2,Intersection,2


In [45]:
# the new size of the dataset:
df.shape

(187525, 7)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187525 entries, 0 to 194672
Data columns (total 7 columns):
SEVERITYCODE    187525 non-null int64
WEATHER         187525 non-null object
ROADCOND        187525 non-null object
LIGHTCOND       187525 non-null object
VEHCOUNT        187525 non-null int64
ADDRTYPE        187525 non-null object
PERSONCOUNT     187525 non-null int64
dtypes: int64(3), object(4)
memory usage: 11.4+ MB


In [53]:
df['SEVERITYCODE','WEATHER','ROADCOND','LIGHTCOND','VEHCOUNT','ADDRTYPE','PERSONCOUNT'].value_counts().to_frame()

,SEVERITYCODE
1,130642
2,56883


In [54]:
df['WEATHER'].value_counts().to_frame()

,WEATHER
Clear,110499
Raining,32976
Overcast,27551
Unknown,14059
Snowing,896
Other,790
Fog/Smog/Smoke,563
Sleet/Hail/Freezing Rain,112
Blowing Sand/Dirt,49
Severe Crosswind,25


In [62]:
filt_wth = {'Clear':1,'Raining':2,'Overcast':3,'Unknown':0,'Snowing':4,'Other':5,'Fog/Smog/Smoke':6,
         'Sleet/Hail/Freezing Rain':7,'Blowing Sand/Dirt':8,'Severe Crosswind':9,'Partly Cloudy':10}
df['WEATHER_X'] = df['WEATHER'].replace(filt_wth)
df['WEATHER_X'].value_counts().to_frame()

,WEATHER_X
1,110499
2,32976
3,27551
0,14059
4,896
5,790
6,563
7,112
8,49
9,25


In [55]:
df['ROADCOND'].value_counts().to_frame()

,ROADCOND
Dry,123736
Wet,47223
Unknown,14009
Ice,1193
Snow/Slush,992
Other,124
Standing Water,111
Sand/Mud/Dirt,73
Oil,64


In [63]:

filt_rd = {'Dry':1,'Wet':2,'Unknown':0,'Ice':3,'Snow/Slush':4,'Other':5,'Standing Water':6,'Sand/Mud/Dirt':7,'Oil':8}
df['ROADCOND_X'] = df['ROADCOND'].replace(filt_rd)
df['ROADCOND_X'].value_counts().to_frame()

,ROADCOND_X
1,123736
2,47223
0,14009
3,1193
4,992
5,124
6,111
7,73
8,64


In [56]:
df['LIGHTCOND'].value_counts().to_frame()

,LIGHTCOND
Daylight,115408
Dark - Street Lights On,48236
Unknown,12599
Dusk,5843
Dawn,2491
Dark - No Street Lights,1526
Dark - Street Lights Off,1184
Other,227
Dark - Unknown Lighting,11


In [64]:

filt_lt = {'Daylight':1,'Dark - Street Lights On':2,'Unknown':0,'Dusk':3,'Dawn':4,'Dark - No Street Lights':5,
         'Dark - Street Lights Off':6,'Other':7,'Dark - Unknown Lighting':8}
df['LIGHTCOND_X'] = df['LIGHTCOND'].replace(filt_lt)
df['LIGHTCOND_X'].value_counts().to_frame()

,LIGHTCOND_X
1,115408
2,48236
0,12599
3,5843
4,2491
5,1526
6,1184
7,227
8,11


In [57]:
df['VEHCOUNT'].value_counts().to_frame()

,VEHCOUNT
2,145668
1,25550
3,12940
4,2418
5,529
0,197
6,144
7,46
8,15
9,9


In [59]:
df['ADDRTYPE'].value_counts().to_frame()

,ADDRTYPE
Block,123321
Intersection,63462
Alley,742


In [73]:
filt_adt = {'Block':1,'Intersection':2,'Alley':3}
df['ADDRTYPE_X'] = df['ADDRTYPE'].replace(filt_adt)
df['ADDRTYPE_X'].value_counts().to_frame()

,ADDRTYPE_X
1,123321
2,63462
3,742


In [76]:
df['PERSONCOUNT'].value_counts().to_frame()

,PERSONCOUNT
2,109569
3,34988
4,14389
1,11582
5,6560
0,5505
6,2695
7,1126
8,533
9,214


In [77]:
df.head()

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND,VEHCOUNT,ADDRTYPE,PERSONCOUNT,WEATHER_X,ROADCOND_X,LIGHTCOND_X,ADDRTYPE_X
0,2,Overcast,Wet,Daylight,2,Intersection,2,3,2,1,2
1,1,Raining,Wet,Dark - Street Lights On,2,Block,2,2,2,2,1
2,1,Overcast,Dry,Daylight,3,Block,4,3,1,1,1
3,1,Clear,Dry,Daylight,3,Block,3,1,1,1,1
4,2,Raining,Wet,Daylight,2,Intersection,2,2,2,1,2


In [78]:
df.dtypes

SEVERITYCODE     int64
WEATHER         object
ROADCOND        object
LIGHTCOND       object
VEHCOUNT         int64
ADDRTYPE        object
PERSONCOUNT      int64
WEATHER_X        int64
ROADCOND_X       int64
LIGHTCOND_X      int64
ADDRTYPE_X       int64
dtype: object

In [79]:
# Separate majority and minority classes

df_majority = df[df['SEVERITYCODE']==1]
df_minority = df[df['SEVERITYCODE']==2]

# downsample mijority class
df_maj_downsample = resample(df_majority,replace=False,n_samples=56883,random_state = 123)
balanced_df = pd.concat([df_maj_downsample,df_minority])

# Display new class counts
balanced_df.SEVERITYCODE.value_counts()

2    56883
1    56883
Name: SEVERITYCODE, dtype: int64

In [80]:
balanced_df.columns


Index(['SEVERITYCODE', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'VEHCOUNT',
       'ADDRTYPE', 'PERSONCOUNT', 'WEATHER_X', 'ROADCOND_X', 'LIGHTCOND_X',
       'ADDRTYPE_X'],
      dtype='object')

## Defining X & Y

In [81]:
X = np.asarray(balanced_df[['VEHCOUNT', 'ADDRTYPE_X', 'PERSONCOUNT', 'WEATHER_X', 'ROADCOND_X', 'LIGHTCOND_X']])
X[0:5]

array([[2, 1, 2, 0, 0, 0],
       [3, 1, 3, 1, 1, 2],
       [2, 1, 3, 1, 1, 1],
       [2, 2, 4, 1, 1, 1],
       [2, 1, 2, 1, 1, 2]])

In [82]:
Y = np.asarray(balanced_df['SEVERITYCODE'])
Y[0:5]

array([1, 1, 1, 1, 1])

## Database Normalization

In [84]:
X = preprocessing.StandardScaler().fit(X.astype(float)).transform(X)
X[0:5]

array([[ 0.08131764, -0.78059527, -0.37908188, -1.61136633, -2.00414872,
        -1.61296557],
       [ 1.7327247 , -0.78059527,  0.32899156, -0.51547385, -0.38450317,
         0.7317909 ],
       [ 0.08131764, -0.78059527,  0.32899156, -0.51547385, -0.38450317,
        -0.44058734],
       [ 0.08131764,  1.24664304,  1.03706501, -0.51547385, -0.38450317,
        -0.44058734],
       [ 0.08131764, -0.78059527, -0.37908188, -0.51547385, -0.38450317,
         0.7317909 ]])

## Train/Test Split of data

In [87]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=42)
print('Train set: ',X_train.shape,Y_train.shape)
print('Test set: ',X_test.shape,Y_test.shape)

Train set:  (79636, 6) (79636,)
Test set:  (34130, 6) (34130,)


## Implementing Logistric Regresion

In [98]:
# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(X, Y)

# predict the response values for the observations in X
logreg.predict(X)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([1, 1, 1, ..., 1, 2, 2])

In [99]:
# store the predicted response values
Y_pred = logreg.predict(X)

# check how many predictions were generated
len(Y_pred)

113766

In [100]:
# compute classification accuracy for the logistic regression model
from sklearn import metrics

print(metrics.accuracy_score(Y, Y_pred))

0.6421338536996993


## Implement K Nearest Neighbours (KNN)

In [105]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=35)
knn.fit(X, Y)
Y_pred = knn.predict(X)
print(metrics.accuracy_score(Y, Y_pred))

0.6341173988713675
